In [ ]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torchvision import models
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from collections import OrderedDict

In [ ]:
#should output the images in 640 x 640
def load_data(data_folder, batch_size, train):
    transform = {
        'train': transforms.Compose(
            [transforms.Resize([224, 224]),
                #transforms.RandomCrop(224),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])]),
        'test': transforms.Compose(
            [transforms.Resize([224, 224]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])])
        }
    data = datasets.ImageFolder(root = data_folder, transform=transform['train' if train else 'test'])
    total_count = len(data)
    # Also you shouldn't use transforms here but below
    train_count = int(0.7 * total_count)
    valid_count = int(0.2 * total_count)
    test_count = total_count - train_count - valid_count
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(data, (train_count, valid_count, test_count))
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True if train else False)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, drop_last = True if train else False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last = True if train else False)
    
    return train_loader, valid_loader, test_loader

In [ ]:
train_loader, valid_loader, test_loader = load_data('dataset', 16, True)

In [ ]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in model.parameters():
    param.requires_grad = False


classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(512, 100)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(100, 5)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.fc = classifier

In [ ]:

criterion = nn.NLLLoss()

optimizer = optim.SGD(model.fc.parameters() , lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
epochs = 5
steps = 0
running_loss = 0
print_every = 1
max_accuracy = 0
for epoch in range(epochs):
    for inputs, labels in train_loader:
        
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in valid_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    if accuracy >= max_accuracy:
                        max_accuracy = accuracy
                        torch.save(model.state_dict(), 'checkpoint.pth')
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(valid_loader):.3f}.. "
                  f"Test accuracy: {accuracy/len(valid_loader):.3f}")
            running_loss = 0
            model.train()
    

Epoch 1/5.. Train loss: 1.609.. Test loss: 1.614.. Test accuracy: 0.263
Epoch 1/5.. Train loss: 1.678.. Test loss: 1.612.. Test accuracy: 0.263
Epoch 1/5.. Train loss: 1.622.. Test loss: 1.602.. Test accuracy: 0.300
Epoch 1/5.. Train loss: 1.628.. Test loss: 1.599.. Test accuracy: 0.287
Epoch 1/5.. Train loss: 1.599.. Test loss: 1.599.. Test accuracy: 0.312
Epoch 1/5.. Train loss: 1.624.. Test loss: 1.590.. Test accuracy: 0.312
Epoch 1/5.. Train loss: 1.555.. Test loss: 1.576.. Test accuracy: 0.325
Epoch 1/5.. Train loss: 1.590.. Test loss: 1.570.. Test accuracy: 0.325
Epoch 1/5.. Train loss: 1.569.. Test loss: 1.565.. Test accuracy: 0.412
Epoch 1/5.. Train loss: 1.585.. Test loss: 1.551.. Test accuracy: 0.475
Epoch 1/5.. Train loss: 1.599.. Test loss: 1.553.. Test accuracy: 0.487
Epoch 1/5.. Train loss: 1.577.. Test loss: 1.547.. Test accuracy: 0.450
Epoch 1/5.. Train loss: 1.522.. Test loss: 1.543.. Test accuracy: 0.463
Epoch 1/5.. Train loss: 1.527.. Test loss: 1.544.. Test accuracy

In [ ]:
list(train_dataset)[0]

Dataset ImageFolder
    Number of datapoints: 434
    Root location: dataset
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
"""for param in model.parameters():
    param.requires_grad = False


classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier"""

In [ ]:
class mask_net(nn.Module):

    def __init__(self):
        super().__init__()
        self.model_ft    = models.resnet18(pretrained=True)
        for param in self.model_ft.parameters():
            param.requires_grad = False
        self.num_ftrs    = self.model_ft.fc.in_features
        classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(self.num_ftrs, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 5)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
        self.model_ft.fc = classifier #nn.Linear(self.num_ftrs, 5)
        
    def forward(self, x):
        out = self.model_ft(x)
        return out




In [ ]:
print(models.resnet18(pretrained=True))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9f1e7bdb-154a-4549-a646-7775bf1dc7a2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>